In [1]:
import re

from sympy import ZZ, simplify, symbols
from sympy.parsing.sympy_parser import parse_expr

a = 612
b = 342
counter = 0

#Θεωρούμε διαίρεση κάθε κλήση της gcd2 εκτός της τελευταίας όπου επιστρέφει αποτέλεσμα.

1.1

In [2]:
def gcd1(a, b):
    global counter
    if b == 0:
        return a
    counter +=1
    if a >= b:
        return gcd1(b, a - b)
    else:
        return gcd1(a, b - a)

In [3]:
print("The gcd({0},{1}) is {2}".format(a,b,gcd1(a, b)))
print("The number of divisions is {0}".format(counter))
counter = 0 # reseting the counter

The gcd(612,342) is 18
The number of divisions is 9


1.2

In [4]:
def gcd2(a, b):
    global counter
    if b == 0:
        return abs(a)
    counter +=1
    r = ZZ.rem(a, b)
    q = ZZ.quo(a, b)
    if r > abs(b) / 2:
        if q > 0:
            q = q + 1
            r = a - b * q
            return gcd2(b, r)
        else:
            q = q - 1
            r = a - b * q
            return gcd2(b, r)
    elif r <= abs(b) / 2:
        return gcd2(b, r)

In [5]:
print("The gcd({0},{1}) is {2}".format(a,b,gcd2(a, b)))
print("The number of divisions is {0}".format(counter))
counter = 0 # reseting the counter

The gcd(612,342) is 18
The number of divisions is 3


Βλέπουμε οτι οι ο αριθμός των διαιρέσεων έγινε 3 φορές μικρότερος

1.4 Extended gcd

1.4.1

In [6]:
def gcd3(a, b, dividend=None, divisor=None, quo=None, rem=None):
    dividend = dividend or []
    divisor = divisor or []
    quo = quo or []
    rem = rem or []

    if b == 0:
        size = len(rem) - 1
        # build a string representation of the equation

        eq_str = "{0}-{1}*{2}".format(dividend[size - 1], divisor[size - 1], quo[size - 1])
        # replace each number until we reach the last two numbers
        for i in reversed(range(2, size)):
            sub_str = "({0}-{1}*{2})".format(dividend[i - 1], divisor[i - 1], quo[i - 1])
            eq_str = re.sub(str(dividend[i + 1]), sub_str, eq_str)

            sub_str = "({0}-{1}*{2})".format(dividend[i - 2], divisor[i - 2], quo[i - 2])
            eq_str = re.sub(str(dividend[i]), sub_str, eq_str)

        # substitute dividend[1] with the variable x
        eq_str = re.sub(str(dividend[1]), "x", eq_str)
        
        # substitute dividend[0] with the variable y
        eq_str = re.sub(str(dividend[0]), "y", eq_str)
        
        # we can now simplify the expression
        final_eq = simplify(parse_expr(eq_str))

        # s is the coefficient of y
        # t is the coefficient of x
        s = final_eq.coeff(symbols("y"))
        t = final_eq.coeff(symbols("x"))
        return s, t, a

    r = ZZ.rem(a, b)
    q = ZZ.quo(a, b)
    
    quo.append(q)
    rem.append(r)
    dividend.append(a)
    divisor.append(b)

    print("{0}={1}*{2}+{3}".format(a,q,b,r))

    return gcd3(b, r, dividend, divisor, quo, rem)

In [7]:
print("The gcd({0},{1}) is {4}, s={2} t={3}".format(a,b,*gcd3(a, b)))

612=1*342+270
342=1*270+72
270=3*72+54
72=1*54+18
54=3*18+0
The gcd(612,342) is 18, s=-5 t=9


1.4.2

In [8]:
def gcd4(a, b, inv=False):
    s0 = 1
    t0 = 0
    s1 = 0
    t1 = 1
    r0 = a
    r1 = b

    q = ZZ.quo(a, b)
    while True:
        r = r0 - r1 * q
        s = s0 - s1 * q
        t = t0 - t1 * q

        s0 = s1
        s1 = s
        t0 = t1
        t1 = t
        r0 = r1
        r1 = r
        if r == 0:
            break
        q = ZZ.quo(b, r)
        b = r

    return s0, t0, r0

In [9]:
print("The gcd({0},{1}) is {4}, s={2} t={3}".format(a,b,*gcd4(a, b)))

The gcd(612,342) is 18, s=-5 t=9


1.5

The argument `positive_result_only` is to return a positive result instead of a negative using

$$-a \pmod{n} \equiv (n-a) \pmod{n}$$

In [10]:
def mod_inverse(a, b, positive_result_only=False):
    # https://math.stackexchange.com/questions/355066/find-the-inverse-modulo-of-a-number-got-a-negative-result
    s, t, d = gcd4(a, b, True)
    if d == 1:
        if positive_result_only and s < 0:
            return b - abs(s)
        else:
            return s
    else:
        return None

In [11]:
m=17
n=3120
print("The inverse modulo of {0} mod {1} is {2}".format(m,n,mod_inverse(m, n)))
print("The positive inverse modulo of {0} mod {1} is {2}".format(m,n,mod_inverse(m, n,True)))

The inverse modulo of 17 mod 3120 is -367
The positive inverse modulo of 17 mod 3120 is 2753


Όλα τα υπολογιστικά αντίγραφα του $Z_{29}$

In [12]:
n=29
m=range(0,29)

for i in m:
    print("The inverse modulo of {0} mod {1} is {2}".format(i,n,mod_inverse(i, n)))


The inverse modulo of 0 mod 29 is None
The inverse modulo of 1 mod 29 is 1
The inverse modulo of 2 mod 29 is -14
The inverse modulo of 3 mod 29 is 10
The inverse modulo of 4 mod 29 is -7
The inverse modulo of 5 mod 29 is 6
The inverse modulo of 6 mod 29 is 5
The inverse modulo of 7 mod 29 is -4
The inverse modulo of 8 mod 29 is 11
The inverse modulo of 9 mod 29 is 13
The inverse modulo of 10 mod 29 is 3
The inverse modulo of 11 mod 29 is 8
The inverse modulo of 12 mod 29 is -12
The inverse modulo of 13 mod 29 is 9
The inverse modulo of 14 mod 29 is -2
The inverse modulo of 15 mod 29 is 2
The inverse modulo of 16 mod 29 is -9
The inverse modulo of 17 mod 29 is 12
The inverse modulo of 18 mod 29 is -8
The inverse modulo of 19 mod 29 is -3
The inverse modulo of 20 mod 29 is -13
The inverse modulo of 21 mod 29 is -11
The inverse modulo of 22 mod 29 is 4
The inverse modulo of 23 mod 29 is -5
The inverse modulo of 24 mod 29 is -6
The inverse modulo of 25 mod 29 is 7
The inverse modulo of 26 